In [ ]:
pip install transformers tensorflow pillow requests matplotlib

from transformers import ViTFeatureExtractor, TFViTForImageClassification
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Google Drive 경로 (data path는 로컬 path 직접 지정해줘야 함)
image_paths = ["/content/drive/MyDrive/cat.png", "/content/drive/MyDrive/dog.png"]

# 이미지 불러오기 및 RGB 변환
images = [Image.open(path).convert("RGB") for path in image_paths]

In [ ]:
# ViT 모델 및 Feature Extractor 불러오기
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = TFViTForImageClassification.from_pretrained("google/vit-base-patch16-224")

In [ ]:
# 이미지 전처리
inputs = feature_extractor(images, return_tensors="tf")  # 전처리된 입력 생성
res = model(**inputs)  # 모델 예측 수행

In [ ]:
# 예측 결과 확인 및 시각화
for i, img in enumerate(images):
    plt.imshow(img)  # 이미지 출력
    plt.xticks([])  # 축 눈금 제거
    plt.yticks([])
    plt.show()
    
    # 예측된 클래스 찾기
    predicted_label = int(tf.math.argmax(res.logits[i], axis=-1))
    prob = float(tf.nn.softmax(res.logits[i])[predicted_label] * 100.0)
    
    # 결과 출력
    print(f"{i}번째 영상의 1순위 분류: {model.config.id2label[predicted_label]} ({prob:.2f}%)")

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image, ImageDraw
import torch
import matplotlib.pyplot as plt

In [ ]:
# DETR 모델 불러오기
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

In [ ]:
# Google Drive 경로 (코랩 환경에서의 예시)
image_paths = ["/content/drive/MyDrive/cat.png", "/content/drive/MyDrive/dog.png"]

# 이미지 불러오기 및 RGB 변환
images = [Image.open(path).convert("RGB") for path in image_paths]

# 이미지 전처리 및 모델 추론
inputs = processor(images, return_tensors="pt")  # PT는 PyTorch 텐서를 의미
outputs = model(**inputs)

# 모델 예측값(바운딩 박스 & 클래스) 가져오기
target_sizes = [image.size[::-1] for image in images]  # 이미지 크기
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)

In [ ]:
def draw_boxes(image, results, threshold=0.9):
    """ 예측된 객체를 시각화하여 출력하는 함수 """
    draw = ImageDraw.Draw(image)
    
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        if score > threshold:  # 신뢰도가 높은 경우만 출력
            box = [round(i, 2) for i in box.tolist()]  # 좌표 변환
            draw.rectangle(box, outline="red", width=3)  # 바운딩 박스 그리기
            text = f"{model.config.id2label[label.item()]}: {score:.2f}"
            draw.text((box[0], box[1]), text, fill="red")  # 클래스명 + 확률 표시
            
    return image

# 각 이미지에 대해 결과 출력
for i, (image, result) in enumerate(zip(images, results)):
    plt.figure(figsize=(8, 6))
    plt.imshow(draw_boxes(image, result))
    plt.axis("off")
    plt.show()